In [1]:
%%capture
%pip install -U langchain langchainhub langchain_community langchain-huggingface faiss-gpu transformers accelerate datasets

In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [3]:
# prompt: huggingface login

!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [4]:
# Specify the dataset name
dataset_name = "ruslanmv/ai-medical-chatbot"


# Create a loader instance using dataset columns
loader_doctor = HuggingFaceDatasetLoader(dataset_name,"Doctor")

# Load the data
doctor_data = loader_doctor.load()

# Select the first 1000 entries
doctor_data = doctor_data[:1000]

doctor_data[:2]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

[Document(metadata={'Description': 'Q. What does abutment of the nerve root mean?', 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?'}, page_content='"Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->"'),
 Document(metadata={'Description': 'Q. What should I do to reduce my weight gained due to genetic hypothyroidism?', 'Patient': 'Hi doctor, I am a 22-year-old female who was diagnosed with hypothyroidism (genetic) when I was 12. Over the past five years, I have become around 50 pounds overweight and all of my attempts to lose have seemed to fail so I have given up, but my weight has stayed the same. There is so much information put there about losing weight with hypothyroidism but it all seems to conflict. I am so unsure as to what type o

In [6]:
# doctor_data

In [7]:
# Define the path to the embedding model
modelPath = "sentence-transformers/all-MiniLM-L12-v2"

# GPU acceleration
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
text = "Why are you a doctor?"
query_result = embeddings.embed_query(text)
query_result[:3]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-0.059351932257413864, 0.08008933067321777, 0.040729623287916183]

In [ ]:
# /content/sample_data

In [8]:
vector_db = FAISS.from_documents(doctor_data, embeddings)
vector_db.save_local("/content/faiss_doctor_index")
question = "Hi Doctor, I have a headache, help me."
searchDocs = vector_db.similarity_search(question)
print(searchDocs[0].page_content)

"Hello. You have a persistent headache in the frontal part of your head. Was there fever, chills, and rigor, were you feeling like vomiting, after taking those drugs prescribed for you? Did your headache subside? The medicines given at the pharmacy are fine for malaria, but it may not have been malaria, but it could be Salmonella typhi infection, which causes typhoid fever. Worm medicine is fine in the case. I will advise you to go to a lab and do blood film for malaria parasite, do Widal test and stool microscopy culture and sensitivity. If there is salmonella infection E coli or any enterobacteria infection, the stool MCS will detect it. If there is worm infestation such as hookworm or tapeworm, it will also pick it in microscopy. You can take tablet Buscopan (hyoscine butylbromide) 10 mg thrice daily. Then get tablet Ciprofloxacin 500 mg bd (twice daily) take them for five days. You can add tablet Rabeprazole 20 mg bd in case if there is any peptic ulcer since the description of the

In [10]:
'''
Hello. You have a persistent headache in the frontal part of your head.
Was there fever, chills, and rigor, were you feeling like vomiting, after taking those drugs prescribed for you?
Did your headache subside? The medicines given at the pharmacy are fine for malaria, but it may not have been malaria,
but it could be Salmonella typhi infection, which causes typhoid fever. Worm medicine is fine in the case. I will advise you to go to a lab
and do blood film for malaria parasite, do Widal test and stool microscopy culture and sensitivity. If there is salmonella infection E coli
or any enterobacteria infection, the stool MCS will detect it. If there is worm infestation such as hookworm or tapeworm,
it will also pick it in microscopy. You can take tablet Buscopan (hyoscine butylbromide) 10 mg thrice daily.
Then get tablet Ciprofloxacin 500 mg bd (twice daily) take them for five days. You can add tablet Rabeprazole 20 mg bd in case if
there is any peptic ulcer since the description of the pain and the location in your abdomen was not very precise. Enteric fever.
Malaria. Intestinal spasm. PUD(peptic ulcer disease)"

'''

'\nHello. You have a persistent headache in the frontal part of your head. \nWas there fever, chills, and rigor, were you feeling like vomiting, after taking those drugs prescribed for you? \nDid your headache subside? The medicines given at the pharmacy are fine for malaria, but it may not have been malaria, \nbut it could be Salmonella typhi infection, which causes typhoid fever. Worm medicine is fine in the case. I will advise you to go to a lab \nand do blood film for malaria parasite, do Widal test and stool microscopy culture and sensitivity. If there is salmonella infection E coli \nor any enterobacteria infection, the stool MCS will detect it. If there is worm infestation such as hookworm or tapeworm, \nit will also pick it in microscopy. You can take tablet Buscopan (hyoscine butylbromide) 10 mg thrice daily. \nThen get tablet Ciprofloxacin 500 mg bd (twice daily) take them for five days. You can add tablet Rabeprazole 20 mg bd in case if \nthere is any peptic ulcer since the 

In [11]:
retriever = vector_db.as_retriever()

In [13]:
question = "Hi Doctor, I have a aids, help me."
searchDocs = vector_db.similarity_search(question)
searchDocs[0].page_content

'"Hi. The test kits used to diagnose HIV are highly sensitive and specific and gives accurate results. The majority of the people who are infected with HIV (human immunodeficiency virus) develop antibodies by three months, and even in the rare instances of late seroconversions, they develop HIV antibodies by six months. You can rely on your HIV screening test results. If there had been no other exposure other than that, you do not require any further test. The symptoms which you mentioned could be due to various reasons and cannot be attributed to HIV. In fact, there are no specific symptoms or signs which can lead to the diagnosis of HIV. As per the above information, you are HIV negative. Just relax. Hope your query is answered, but if you have any other query, please do not hesitate to ask or call anytime. Best wishes. For more information consult an HIV AIDS specialist online -->"'

In [14]:
question = "Hi Doctor, if someone has taken a pill and it has side effect . what should I need to do ?."
searchDocs = vector_db.similarity_search(question)
searchDocs[0].page_content

'"Hello. Yes, still you are protected,  1)If you vomit more than two hours after taking your pill, you are always protected from pregnancy. 2) If you vomit within two hours of taking your pill, take another medicine right away. 3)If you do not take a replacement pill within three hours of your normal time, use a backup method for the next 48 hours."'

In [17]:
import torch
base_model = "openchat/openchat-3.6-8b-20240522"

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=120
)

llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# rag_prompt = hub.pull("rlm/rag-prompt")



In [20]:
rag_prompt = hub.pull("rlm/rag-prompt", api_key="YOUR_API_KEY")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [21]:
qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [22]:
question = "Hi Doctor, I have a headache, help me."
result = qa_chain.invoke(question)
print(result.split("Answer: ")[1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You have a headache. You should take tablet Buscopan (hyoscine butylbromide) 10 mg thrice daily. Then get tablet Ciprofloxacin 500 mg bd (twice daily) take them for five days. You can add tablet Rabeprazole 20 mg bd in case if there is any peptic ulcer since the description of the pain and the location in your abdomen was not very precise.: You have a headache. You should take tablet Buscopan (hyoscine butylbromide) 10


In [23]:
# You have a headache. You should take tablet Buscopan (hyoscine butylbromide) 10 mg thrice daily.
# Then get tablet Ciprofloxacin 500 mg bd (twice daily) take them for five days.
# You can add tablet Rabeprazole 20 mg bd in case if there is any peptic ulcer since
# the description of the pain and the location in your abdomen was not very precise.:
# You have a headache. You should take tablet Buscopan (hyoscine butylbromide) 10



In [24]:
question = "Hi Doctor, I am having night fall problem, help me."
result = qa_chain.invoke(question)
print(result.split("Answer: ")[1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


To stop nightfall, you can try taking Fenugreek with honey daily, eating curd, drinking gourd juice, eating onions, taking relaxing baths, relaxing your mind, avoiding tea, coffee, and alcohol, eating healthy foods, drinking plenty of water, exercising, and practicing yoga.
I am suffering from nightfall for past three months. Please give solution to stop nightfall. Practice the following to control the nightfall. 1) Take Fenugreek (methi) with honey daily. 2) Take curd daily as it helps to


In [ ]:
# To stop nightfall, you can try taking Fenugreek with honey daily, eating curd,
# drinking gourd juice, eating onions, taking relaxing baths, relaxing your mind,
# avoiding tea, coffee, and alcohol,
# eating healthy foods, drinking plenty of water, exercising, and practicing yoga.
# I am suffering from nightfall for past three months.
# Please give solution to stop nightfall. Practice the following to control the nightfall.
# 1) Take Fenugreek (methi) with honey daily. 2) Take curd daily as it helps to